## runnableLambda 객체 사용방법 실습

In [21]:
from dotenv import load_dotenv
import os
load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# GEMINI_API_KEY

In [2]:
from langchain_core.runnables import RunnableLambda

In [3]:
# 함수 정의
def func(x):
    return  x*2


In [4]:
# 함수 전달인자로 func 넣기

r1 = RunnableLambda(func)

In [5]:
input_data = 5
r1.invoke(input_data)

10

In [6]:
r1 = RunnableLambda(lambda x: x*2)
r1.invoke(5)

10

In [7]:
# runnable batch

r3 = RunnableLambda(func)

input_data = [5, 10, 20]

r3.batch(input_data)

[10, 20, 40]

### 순차적으로 실행하기

In [8]:
from langchain_core.runnables import RunnableLambda

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

chain = r1 | r2

chain.invoke(20)

65

In [10]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# chain = r1 | r2  # 아래와 같은 표현
chain = RunnableSequence(r1, r2)

chain.invoke(20)

65

In [11]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

r1 = RunnableLambda(lambda x: 3 * x)
r2 = RunnableLambda(lambda x: x + 5)

# 병렬처리 , r1, r2 키는 임으로 설정해 주면 됨.
# chain = RunnableParallel(r1=r1, r2=r2)
chain = RunnableParallel(first=r1, second=r2)

chain.invoke(10)

{'first': 30, 'second': 15}

In [13]:
!pip install -qU grandalf

In [14]:
[{"foo":2}]*3

[{'foo': 2}, {'foo': 2}, {'foo': 2}]

In [15]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable_1 = RunnableLambda(lambda x: {"foo": x})
runnable_2 = RunnableLambda(lambda x: [x] * 3)
runnable_3 = RunnableLambda(lambda x: str(x))

chain = runnable_1 | RunnableParallel(r2=runnable_2, r3=runnable_3)

print(chain.invoke(2))
chain.get_graph().print_ascii()

{'r2': [{'foo': 2}, {'foo': 2}, {'foo': 2}], 'r3': "{'foo': 2}"}
        +-------------+        
        | LambdaInput |        
        +-------------+        
               *               
               *               
               *               
          +--------+           
          | Lambda |           
          +--------+           
               *               
               *               
               *               
   +----------------------+    
   | Parallel<r2,r3>Input |    
   +----------------------+    
          *         *          
        **           **        
       *               *       
+--------+          +--------+ 
| Lambda |          | Lambda | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
  +-----------------------+    
  | Parallel<r2,r3>Output |    
  +-----------------------+    


## LCEL 문법 적용

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                        temperature=0.2,
                        google_api_key = GEMINI_API_KEY)

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

content='주어진 컨텍스트("Vector Search에 의한 컨텐스트 내용")는 질문 "안녕"에 대한 답변을 포함하고 있지 않습니다.\n\n따라서 해당 컨텍스트 내에서는 "안녕"이라는 질문에 답할 수 없습니다.' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019b8d10-ce2f-73a0-8b56-781488709e64-0' usage_metadata={'input_tokens': 39, 'output_tokens': 556, 'total_tokens': 595, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 505}}


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

chain = prompt | llm
input = {
            "context": "Vector Search에 의한 컨텐스트 내용", 
            "query": "안녕"
        }
print(chain.invoke(input))

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

prompt = ChatPromptTemplate.from_template("""
    다음 컨텍스트에 대해서만 답하세요.
    컨텍스트 :
    {context}
    질문:
    {query}
""")

# 기본값을 제공하는 함수
def add_default_values(input_dict):
    return {
        "context": input_dict.get("context", "너는 Langchain에 대해 설명해주는 AI전문 강사야"),
        "query": input_dict.get("query", "나는 왜 langchain이 개발되었는지 궁금해")
    }


# 체인 구성
chain = RunnableLambda(add_default_values) | prompt | llm

In [33]:
# 빈 딕셔너리로도 실행 가능
result = chain.invoke({})
print(result)

content='아주 좋은 질문입니다! 왜 LangChain이 개발되었는지 이해하는 것이 이 프레임워크의 본질을 파악하는 데 가장 중요합니다.\n\n자, 제가 AI 전문 강사의 입장에서 명확하게 설명해 드릴게요.\n\nLLM(대규모 언어 모델)은 그 자체로 엄청나게 강력한 \'두뇌\'입니다. 자연어를 이해하고, 생성하고, 다양한 지식과 추론 능력을 가지고 있죠. 하지만 이 두뇌를 현실 세계의 다양한 정보와 연결하고, 특정 작업을 수행하도록 지시하며, 여러 단계를 거쳐 복잡한 문제를 해결하게 만드는 것은 별개의 이야기였습니다.\n\nLangChain이 개발된 주요 이유는 다음과 같은 "갭(Gap)"을 채우기 위함입니다:\n\n1.  **LLM의 한계:**\n    *   **외부 정보의 부재:** LLM은 훈련받은 데이터 내의 지식만 가지고 있습니다. 실시간 웹 정보, 기업 내부 데이터베이스, 사용자의 개인 파일 등 외부의 최신 정보를 직접 조회하거나 활용할 수 없습니다.\n    *   **도구 사용의 어려움:** LLM은 계산기, 검색 엔진, API 호출 등 특정 기능을 수행하는 외부 \'도구\'를 직접 사용할 수 없습니다. 개발자가 수동으로 LLM의 응답을 파싱하고, 도구를 호출하고, 그 결과를 다시 LLM에게 넘겨주는 복잡한 과정을 거쳐야 했습니다.\n    *   **다단계 작업의 복잡성:** LLM은 한 번의 질문에 답하는 데는 능숙하지만, 여러 단계를 거쳐야 하는 복잡한 작업 (예: "이메일 초안 작성 후 관련 정보 검색해서 첨부해줘")을 수행하게 하는 것은 어려웠습니다. 각 단계를 수동으로 연결해야 했죠.\n    *   **기억 (Memory)의 부재:** LLM은 기본적으로 \'상태 비저장(stateless)\'입니다. 이전 대화의 맥락을 기억하지 못하므로, 장기적인 대화를 이어가기 위해서는 개발자가 직접 대화 이력을 관리하고 프롬프트에 주입해야 했습니다.\n\n2.  **개발 효율성의 문제:**\n    *   위와 같은 문제를 해결하기 위해

In [36]:
from IPython.display import display, Markdown


display(Markdown(result.content))

아주 좋은 질문입니다! 왜 LangChain이 개발되었는지 이해하는 것이 이 프레임워크의 본질을 파악하는 데 가장 중요합니다.

자, 제가 AI 전문 강사의 입장에서 명확하게 설명해 드릴게요.

LLM(대규모 언어 모델)은 그 자체로 엄청나게 강력한 '두뇌'입니다. 자연어를 이해하고, 생성하고, 다양한 지식과 추론 능력을 가지고 있죠. 하지만 이 두뇌를 현실 세계의 다양한 정보와 연결하고, 특정 작업을 수행하도록 지시하며, 여러 단계를 거쳐 복잡한 문제를 해결하게 만드는 것은 별개의 이야기였습니다.

LangChain이 개발된 주요 이유는 다음과 같은 "갭(Gap)"을 채우기 위함입니다:

1.  **LLM의 한계:**
    *   **외부 정보의 부재:** LLM은 훈련받은 데이터 내의 지식만 가지고 있습니다. 실시간 웹 정보, 기업 내부 데이터베이스, 사용자의 개인 파일 등 외부의 최신 정보를 직접 조회하거나 활용할 수 없습니다.
    *   **도구 사용의 어려움:** LLM은 계산기, 검색 엔진, API 호출 등 특정 기능을 수행하는 외부 '도구'를 직접 사용할 수 없습니다. 개발자가 수동으로 LLM의 응답을 파싱하고, 도구를 호출하고, 그 결과를 다시 LLM에게 넘겨주는 복잡한 과정을 거쳐야 했습니다.
    *   **다단계 작업의 복잡성:** LLM은 한 번의 질문에 답하는 데는 능숙하지만, 여러 단계를 거쳐야 하는 복잡한 작업 (예: "이메일 초안 작성 후 관련 정보 검색해서 첨부해줘")을 수행하게 하는 것은 어려웠습니다. 각 단계를 수동으로 연결해야 했죠.
    *   **기억 (Memory)의 부재:** LLM은 기본적으로 '상태 비저장(stateless)'입니다. 이전 대화의 맥락을 기억하지 못하므로, 장기적인 대화를 이어가기 위해서는 개발자가 직접 대화 이력을 관리하고 프롬프트에 주입해야 했습니다.

2.  **개발 효율성의 문제:**
    *   위와 같은 문제를 해결하기 위해 개발자들은 매번 복잡한 파이프라인을 직접 코딩해야 했습니다. 이는 시간 소모적이고, 오류 발생 가능성이 높으며, 재사용하기 어려운 코드를 양산했습니다.

**결론적으로, LangChain은 LLM을 활용한 애플리케이션 개발을 위한 일종의 '운영 체제' 또는 '프레임워크' 역할을 합니다.**

LLM을 단순히 텍스트를 생성하는 도구를 넘어, 외부 환경과 상호작용하고, 기억하며, 능동적으로 문제를 해결하는 지능형 시스템으로 변모시키는 다리 역할을 하는 것이죠. LangChain은 이 과정을 쉽게 만들어주는 모듈화된 구성 요소와 표준화된 인터페이스를 제공하여, 개발자들이 LLM의 잠재력을 최대한 발휘하는 강력하고 복잡한 애플리케이션을 더 쉽고 빠르게 구축할 수 있도록 돕기 위해 탄생했습니다.

이해되셨나요? LangChain 덕분에 우리는 LLM으로 훨씬 더 역동적이고 유용한 것들을 만들 수 있게 된 겁니다.

In [30]:
result2 = chain.invoke({"query": "RunnableLambda에 대해 1문장으로 설명해줘"})

In [31]:
print(result2.content)

제공된 컨텍스트에는 RunnableLambda에 대한 정보가 없습니다.
